# Context-specific metabolic networks from omics

Here we will illustrate the use of the iMAT method, as implemented in CORNETO, to extract context specific metabolic networks from transcriptomics. CORNETO implements an extension of iMAT to support multiple samples, and also gives more control for customization.

We will use the core [E. coli PKN](https://systemsbiology.ucsd.edu/Downloads/E_coli_Core), and microarray-based gene [expression data]( http://systemsbiology.ucsd.edu/InSilicoOrganisms/Ecoli/EcoliExpression2) corresponding to a anaerobic growth on glucose of a wild-type E. coli strain. This tutorial is based on the original [COBRA tutorial](https://opencobra.github.io/cobratoolbox/stable/tutorials/tutorial_extractionTranscriptomic.html).

In [ ]:
import corneto as cn
import pandas as pd
import numpy as np

cn.info()

## Loading data from COBRA

We will use the ecoli core metabolic network and ecoli data example obtained from the COBRA package

In [ ]:
from corneto.io import import_cobra_model

G = import_cobra_model("../../tests/gem/ecoli_core.zip")
df_expr = pd.read_csv("../../tests/gem/ecoli_example_data.zip")
df_expr

In [ ]:
G.shape

In [ ]:
# We can inspect the content of the network
G.get_attr_edges()[:5]

## Mapping data using Gene-Protein-Reaction (GPR) rules

We need to compute reaction scores from gene expression data. For this, we will classify genes as highly expressed and lowly expressed based on thresholds applied to the marginal distribution of gene intensities. Then, we will map these values by applying the Gene-Protein-Reaction (GPR) rules included in the Ecoli PKN.

In [ ]:
from corneto.methods.metabolism import evaluate_gpr_expression, get_unique_genes

genes = get_unique_genes(G, startswith="b")
len(genes)

In [ ]:
df_expr.set_index("gene").stack().to_frame(name='expression').hist(bins=30);

In [ ]:
df_expr["high"] = (df_expr["value"] >= 8).astype(int)
df_expr["low"] = (df_expr["value"] <= 4).astype(int)
df_expr["score"] = df_expr["high"] - df_expr["low"]
df_expr

In [ ]:
values = df_expr[["gene", "score"]].set_index("gene").to_dict()["score"]
e = evaluate_gpr_expression(G.get_attr_from_edges("GPR"), values)

In [ ]:
pd.DataFrame({"id": G.get_attr_from_edges("id"), "score": e})

## Context-specific network inference

We will use the reaction scores to create a `Data` object. The `id` corresponds to the reaction id of the PKN, and the value to the reaction score. We indicate that these features map to edges (reactions) with `mapping` = `edge`. Note that if `mapping` = `none`, then it will be assumed that the dataset contains genes, and automatic GPR will be performed to translate from genes to reaction scores.

In [ ]:
# Create a dataset (here we only have 1 sample, which is the mean of the four measurements)
from corneto._data import Data

feats = dict()
for edge, val in zip(G.get_attr_from_edges("id"), e):
    feats[edge] = {"value": val, "mapping": "edge"}
data = {
    "sample1": feats
}
data = Data.from_cdict(data)
data

We run the iMAT method (in this case, only 1 sample, which is the mean of the 4 biological replicates)

In [ ]:
from corneto.methods.future.imat import MultiSampleIMAT

# eps is the min absolute flux value that we will consider
# for reactions with positive scores, i.e., if a reaction
# with a positive score is included in the context specific
# metabolic network, it has to be able to carry an absolute
# metabolic flux value above eps.
eps = 1e-2

# We will also add a small penalty to make networks sparser.
# When we have more than 1 sample, this parameter controls 
# the regularization across samples, blocking entire groups
# of reactions across samples. Here, we are just penalizing
# the inclusion of reactions not needed to fit the reaction
# scores
m = MultiSampleIMAT(lambda_reg=1e-3, eps=eps)
P = m.build(G, data)
P.expr

In [ ]:
P.solve(solver="scipy", verbosity=0);

The final result indicates that 1 reaction with positive score was not selected, and 1 reaction with negative score was included. In total, the network contains 61 reactions with fluxes. The last value corresponds to the number of reactions that were not blocked.

In [ ]:
for o in P.objectives:
    print(o.value)

In [ ]:
# Metabolic fluxes correspond to values of the flow variable
# Note that there are 91 values, corresponding to the 91 reactions
# in the PKN

df_sol = pd.DataFrame(
    P.expr.flow.value, index=G.get_attr_from_edges("id"), columns=["flux"]
)
df_sol

In [ ]:
df_sol.hist(bins=30)

In [ ]:
selected_reactions = df_sol[df_sol.flux.abs() >= eps*(1-eps)].index.values.tolist()
len(selected_reactions)

In [ ]:
np.sum(P.expr.edge_has_flux.value)

In [ ]:
# Extract the context specific network
G_ctx = G.edge_subgraph(np.flatnonzero(P.expr.edge_has_flux.value))
G_ctx.shape

In [ ]:
G_ctx.plot(layout="neato")

## Old version

Original experiments in the manuscript used the old `multicondition_imat`:

In [ ]:
from corneto.methods.metabolism.fba import multicondition_imat

P = multicondition_imat(G, np.array(e), alpha=1e-3, eps=eps)
P.solve(solver="scipy");

In [ ]:
for o in P.objectives:
    print(o.value)